## basics

- policy function: $\pi(a|s)$ is a PDF
    - $\sum_{a}\pi(a|s)=1$
- Policy Network: $\pi(a|s;\mathbf \theta)$
    - $\bf\theta$ 表示神经网络的待训练参数
    - $\sum_{a}\pi(a|s;\theta)=1$（last layer of nn，称之为 logits，接一个 softmax 的变换）
- 状态价值函数（state value function），对其做函数近似（function approximation）；

    - **Discounted reward**

        $$
        U_t=R_t+\gamma R_{t+1} + \gamma^2R_{t+2} + \gamma^3R_{t+3}+\cdots
        $$

        - $U_t$ 的精确计算依赖 $A_t,A_{t+1},A_{t+2}, \cdots, $ 和 $S_t,S_{t+1},S_{t+2},\cdots$
        - $t$ 时刻，未来的 reward 为还未观测到的随机变量，
            - 每个奖励 $R_t$ 的随机性都来自于前一个时刻的动作 $A_t$ 和 $S_t$
            - 动作的随机性来自于策略函数 $\pi(a|\cdot)$，状态的随机性来自于状态转移函数 $p(s_t|s_{t-1})$
    - **action-value function**
        $$
        Q_\pi(s_t,a_t)=\mathbb E[U_t|S_t=s_t,A_t=a_t]
        $$

### 策略学习的目标函数

- $S_t,A_t,S_{t+1},A_{t+1}, S_{t+2}, A_{t+2}$
- $Q_\pi(s_t,a_t)=\mathbb E[U_t|S_t=s_t,A_t=a_t]$
- $V_\pi(s_t)=\mathbb E_{A_t\sim \pi(\cdot|s_t;\theta)}Q_\pi(s_t,A_t)$
- 状态价值（state value）既依赖当前状态 $s_t$ 也依赖策略网络 $\pi$ 的参数 $\theta$
    - 当前状态 $s_t$ 越好，$V_\pi(s_t)$ 越大，回报 $U_t$ 的期望也就越大；
    - 策略网络 $\pi$ 越好，$V_\pi(s_t)$ 也会越大；
- 定义目标函数

    $$
    \begin{split}
    &J(\theta)=\mathbb E_S[V_\pi(S)]\\
    &\max_{\theta} J(\theta)
    \end{split}
    $$
    
    - 最大化目标，通过梯度上升来优化
    
    $$
    \theta\leftarrow \theta+\eta \frac{\partial J(\theta)}{\partial \theta}
    $$

### 策略梯度定理

$$
\frac{\partial J(\theta)}{\partial \theta}=\mathbb E_S\left[\mathbb E_{A\sim \pi(\cdot|S;\theta)}\left[\frac{\partial \ln_\pi(A|S;\theta)}{\partial \theta}\cdot Q_\pi(S,A)\right]\right]
$$